# Segmenting and Clustering in Toronto
This notebook is for submission to the IBM Data Science Professional Certificate capstone, Week 3 assignment. Data will be scraped from the Toronto Wikipedia page.

## Section 1: Scraping the data, creating the dataframe, and cleaning the data.
### Scraping the data from the website.

In [11]:
import pandas as pd

# Site URL
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

# Write to dataframe
df = pd.read_html(url, header=0)

In [4]:
len(df)

3

This implies that pandas read three tables from this url. Upon inspection, we see that we want the first table (index=0).

In [10]:
df[0].head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


We define our dataframe to be the first table.

In [12]:
df = df[0]
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [13]:
df.shape

(180, 3)

### Cleaning and formatting the data.

In [14]:
df.groupby('Borough').count()

,Postal Code,Neighborhood
Borough,,
Central Toronto,9,9
Downtown Toronto,19,19
East Toronto,5,5
East York,5,5
Etobicoke,12,12
Mississauga,1,1
North York,24,24
Not assigned,77,77
Scarborough,17,17


There are 77 `Not assigned` boroughs. We drop those from the dataframe.

In [34]:
df = df[df['Borough'] != 'Not assigned']

df.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [35]:
# The shape of the resulting dataframe
df.shape

(103, 3)

##